In [2]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# MongoDB Atlas credentials
user_name = os.getenv('MONGO_DB_USER_NAME')
db_password = os.getenv('MONGO_DB_PASSWORD')
cluster_url = os.getenv('MONGO_DB_CLUSTER_URL')

# Construct the MongoDB URI
uri = f"mongodb+srv://{user_name}:{db_password}@{cluster_url}/?retryWrites=true&w=majority&appName=logistics-cluster"

# Connect to MongoDB
client = MongoClient(uri, server_api=ServerApi('1'))

# Select the database and collection
db = client['your_database_name']
collection = db['your_collection_name']

# Fetch data from the collection
try:
    # Query all documents in the collection
    documents = collection.find()

    # Print each document
    for doc in documents:
        print(doc)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the connection
    client.close()

{'_id': ObjectId('67c1896b4f92b1e897db6e8f'), 'equipmentEvent': {'documentReferences': [{'documentReferenceType': 'BL', 'documentReferenceValue': '33633dbd-2864-4800-9ec6-acd905d1b0bc'}], 'equipmentEventTypeCode': 'ARRIVAL', 'equipmentReference': 'APZU6551621', 'eventClassifierCode': 'ACT', 'eventCreatedDateTime': '2024-04-29T08:12:59.209045', 'eventDateTime': '2024-04-29T08:12:59.209045', 'eventID': 'd0d66b8e-9710-44a8-9387-d88eed27bbf1', 'eventLocation': {'address': {'city': 'København', 'country': 'France', 'floor': '5F', 'name': 'Henrik', 'postCode': '8495', 'stateRegion': 'N/A', 'street': 'Main Street', 'streetNumber': '55'}, 'facilityCode': 'PQR', 'facilityCodeListProvider': 'SMDG', 'latitude': '-49.607455', 'longitude': '23.742359', 'locationName': 'Port Terminal', 'unlocationCode': 'XYZ123'}, 'eventType': 'EQUIPMENT', 'seals': [{'sealNumber': '57d513fa-14a7-4155-982b-bdea75152ed6', 'sealSource': 'CUSTOMS', 'sealType': 'STANDARD'}], 'transportCall': {'carrierServiceCode': 'FE1',

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# Read data from a collection
